# [CSC8101] Engineering for AI - 2024 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
  - actions to get around the lazy execution of spark: 
  https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/mutazb999/CSC8101-lab-and-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    subest_trips = df[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']]
    # input: trips dataset
    df = df.filter(~((subest_trips.trip_distance == 0) | ((subest_trips.passenger_count == 0) & (subest_trips.total_amount == 0))))

    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,858,164


In [0]:
from pyspark.sql.functions import expr, percentile_approx, abs

# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # `total_amount`
    # Calculating z-score on `total_amount` (ta_*) columns
    ta_median_x = df.selectExpr('percentile_approx(total_amount, 0.5) as ta_median_x').first().ta_median_x
    ta_median_abs = df.selectExpr('percentile_approx(abs(total_amount - {}), 0.5) as ta_median_abs'.format(ta_median_x)).first().ta_median_abs

    ta_mad_value = 1.483 * ta_median_abs

    # Calculating z_score for `total_amount`
    z_score_expr = expr('(total_amount - {}) / {}'.format(ta_median_x, ta_mad_value))
    df = df.filter(abs(z_score_expr) <= 3.5)

    # `trip_distance`
    # Calculating z-score on `trip_distance` (td_*) columns
    td_median_x = df.selectExpr('percentile_approx(trip_distance, 0.5) as td_median_x').first().td_median_x
    td_median_abs = df.selectExpr('percentile_approx(abs(trip_distance - {}), 0.5) as td_median_abs'.format(td_median_x)).first().td_median_abs

    td_mad_value = 1.483 * td_median_abs

    # Calculating z_score for `total_distance
    z_score_expr = expr('(trip_distance - {}) / {}'.format(td_median_x, td_mad_value))
    df = df.filter(abs(z_score_expr) <= 3.5)

    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,437,424


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitability = total_amount / trip_distance`.

In [0]:
# from pyspark.sql.functions import col

# # Compute unit profitability
# trips_12_with_unit_profitability = trips_12_with_zone_names.withColumn("unit_profitability", col("total_amount") / col("trip_distance"))

# display(trips_12_with_unit_profitability.select("start_zone_name", "end_zone_name", "unit_profitability", "total_amount", "trip_distance"))

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    zone_names_start = zone_names.withColumnRenamed("Zone", "start_zone_name")
    zone_names_end = zone_names.withColumnRenamed("Zone", "end_zone_name")

    # Join the datasets using the aliased names
    df = df.join(
        zone_names_start,
        df["PULocationID"] == zone_names_start["LocationID"],
        "inner"
    ).join(
        zone_names_end,
        df["DOLocationID"] == zone_names_end["LocationID"],
        "inner"
    )

    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 2,437,424


In [0]:
from pyspark.sql.functions import col

# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1    
    # Compute unit profitability
    df = df.withColumn("unit_profitability", col("total_amount") / col("trip_distance"))

    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 2,437,424


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z1 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitability)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# trips_22.printSchema()

In [0]:
display(trips_22[['PULocationID', 'DOLocationID', 'total_amount', 'trip_distance', 'passenger_count']])

PULocationID,DOLocationID,total_amount,trip_distance,passenger_count
90,68,8.8,0.8,1.0
113,90,8.8,0.9,1.0
88,232,13.8,2.8,1.0
79,249,12.3,1.4,1.0
142,238,12.3,2.0,0.0
114,90,12.8,1.6,1.0
90,144,13.3,1.8,1.0
114,48,14.75,2.0,1.0
48,152,22.3,5.7,1.0
234,148,14.75,1.8,1.0


In [0]:
# # develop your solution here (create/destroy cells as needed) and then implement it in the functions below
# from pyspark.sql.functions import mean, sum, count

# # Summarize interzonal travel
# interzonal_travel = trips_22.groupby("PULocationID", "DOLocationID", "start_zone_name", "end_zone_name").agg(
#     mean("unit_profitability").alias("average_unit_profit"),
#     count("trip_distance").alias("trips_count"),
#     sum("passenger_count").alias("total_passengers")
# )


# interzonal_travel = interzonal_travel.select(
#     "PULocationID",
#     "DOLocationID",
#     "start_zone_name",
#     "end_zone_name",
#     "average_unit_profit",
#     "trips_count",
#     "total_passengers"
# )

# display(interzonal_travel)

In [0]:
# from pyspark.sql.functions import desc

# # Obtain top-10 zones based on different measures
# top_10_trips = interzonal_travel.groupby("start_zone_name").agg(sum("trips_count").alias("total_trips")).orderBy(desc("total_trips")).limit(10)
# top_10_profitability = interzonal_travel.groupby("start_zone_name").agg(mean("average_unit_profit").alias("average_profitability")).orderBy(desc("average_profitability")).limit(10)
# top_10_passengers = interzonal_travel.groupby("start_zone_name").agg(sum("total_passengers").alias("total_passengers")).orderBy(desc("total_passengers")).limit(10)

# # Print the top-10 zones for each measure
# print("Top 10 zones based on total trips:")
# display(top_10_trips)

# print("Top 10 zones based on average profitability:")
# display(top_10_profitability)

# print("Top 10 zones based on total passengers:")
# display(top_10_passengers)

In [0]:
from pyspark.sql.functions import mean, sum, count

## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    # Summarize interzonal travel
    df = df.groupby("PULocationID", "DOLocationID", "start_zone_name", "end_zone_name").agg(
        mean("unit_profitability").alias("average_unit_profit"),
        count("trip_distance").alias("trips_count"),
        sum("passenger_count").alias("total_passengers")
    )


    df = df.select(
        "PULocationID",
        "DOLocationID",
        "start_zone_name",
        "end_zone_name",
        "average_unit_profit",
        "trips_count",
        "total_passengers"
    )
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
# display(graph.take(10))

Row count: 11,869


In [0]:
# # Obtain top-10 zones based on different measures
# top_10_trips = interzonal_travel.groupby("start_zone_name").agg(sum("trips_count").alias("total_trips")).orderBy(desc("total_trips")).limit(10)
# top_10_profitability = interzonal_travel.groupby("start_zone_name").agg(mean("average_unit_profit").alias("average_profitability")).orderBy(desc("average_profitability")).limit(10)
# top_10_passengers = interzonal_travel.groupby("start_zone_name").agg(sum("total_passengers").alias("total_passengers")).orderBy(desc("total_passengers")).limit(10)

In [0]:
from pyspark.sql.functions import desc

# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = df.groupby("start_zone_name").agg(
        sum("trips_count").alias("total_trips"), 
        mean("average_unit_profit").alias("average_profitability"),
        sum("total_passengers").alias("total_passengers")
    )

    df.collect()

    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(desc("total_trips")).limit(10)
    return df_zones  # it was `None`

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(desc("average_profitability")).limit(10)
    return df_zones  # it was `None`

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(desc("total_passengers")).limit(10)
    return df_zones  # it was `None`

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_trips

DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double]

In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
top10_profit

DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double]

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_passenger

DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double]

## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: XXL
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-XXL-delta/
      Number of trips (dataset rows): 132,396,785
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

[DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double],
 DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double],
 DataFrame[start_zone_name: string, total_trips: bigint, average_profitability: double, total_passengers: double]]

In [0]:
## ASK DEMONSTARTOR IS ROWS CORRESPOND TO THE SIZE OF THE DATASET OR THE BY-PRODUCT OF THE PIPELINE NUMBER OF ROW?
## W/O 1.2 GIVES A CONSISTENT 0.28s TO 0.33s EXECUTION TIME !! IS THAT WHAT IT IS EXPECTED?
sec = 2.77
print(round((sec * (1_000_000 / trips.count())), 3))

0.021


_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898033 |  15.571166 |  41.953716 |  90.443069 |  132.396785 |
| execution time   (w/o 1.2)  | 1.29s | 2.39s | 3.73s | 4.92s | 6.01s |
| execution time              | 1.45s | 6.52s | 25.63s | 35.63s | 39.83s |
| sec / 1M records (w/o 1.2)  | 0.445s  | 0.153s  | 0.089s  | 0.054s  | 0.045s  |
| sec / 1M records            | 0.5s  | 0.419s  | 0.611s  | 0.394s  | 0.301s  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898033 |  15.571166 |  41.953716 |  90.443069 |  132.396785 |
| execution time   (w/o 1.2)  | 0.51s | 0.68s | 0.88s | 1.37s | 2.77s |
| execution time              | 1.49s | 2.62s | 5.95s | 14.38s | 21.51s |
| sec / 1M records (w/o 1.2)  | 0.176s  | 0.044s  | 0.021s  | 0.015s  | 0.021s  |
| sec / 1M records            | 0.514s  | 0.168s  | 0.142s  | 0.159s  | 0.14s  |

Cluster used: CSC8641